# Clean-up Resource

### SageMaker Experiments

In [ ]:
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [ ]:
experiment_name='xgboost-poc-1'  #<== 생성한 experiment_name을 넣어야 합니다.

In [ ]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)

## S3 버킷 삭제

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.session.Session()

In [ ]:
bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
!aws s3 rm s3://$bucket/xgboost